## Initialize credentials to access migdar data

It's recommended to start your notebook like this to  set the env vars:

```
env MIGDAR_USERNAME="" MIGDAR_PASSWORD="" jupyter lab
```

In [1]:
MIGDAR_PASSWORD = %env MIGDAR_PASSWORD
MIGDAR_USERNAME = %env MIGDAR_USERNAME

assert len(MIGDAR_USERNAME) > 5 and len(MIGDAR_PASSWORD) > 5

#### Create an quthorized requests session

In [7]:
import requests

migdar_session = requests.Session()
migdar_session.auth = (MIGDAR_USERNAME, MIGDAR_PASSWORD)

## Download the aggregated search results before moving to the internal search app

In [3]:
%%bash -s "$MIGDAR_USERNAME" "$MIGDAR_PASSWORD"
MIGDAR_USERNAME="${1}"
MIGDAR_PASSWORD="${2}"

# set to "-0" to download all rows (It's 470MB)
LIMIT_ROWS="3000"

mkdir -p data/search_results
wget -qO /dev/stdout \
     --http-user=${MIGDAR_USERNAME} --http-password=${MIGDAR_PASSWORD} \
     https://migdar-internal-search.odata.org.il/__data/search_results/unique_records.csv \
| head -n${LIMIT_ROWS} \
> data/search_results/unique_records.csv

In [4]:
from dataflows import Flow, load, printer, dump_to_path
from collections import deque

def head(rows):
    for rownum, row in enumerate(rows):
        if rownum >= 2:
            break
        yield row

Flow(load('data/search_results/unique_records.csv'), head,
     printer(tablefmt='html', fields=['title', 'pubyear', 'migdar_id'])).process()[1]

#,title (string),pubyear (string),migdar_id (integer)
1,הוראות בלתי שגרתיות לתפירת צואר הרחם,1981,0
2,שינויים בזרימת הדם בשורר הרחם בעקבות הזרקת פרוסטגלנדין ALPHA F2 להפסקת הריון בטרימסטר שני,1984,1


{}

## Load the data from GDrive

These are the column names we store from the user-entered xlsx files on GDrive

In [3]:
SEARCH_IMPORT_FIELD_NAMES = ['Life Domains', 'Resource Type', 'Item Type', 'title', 'pubyear', 'publisher', 'author', 
                             'language_code', 'custom_metadata', 'publication_distribution_details', 'notes', 'tags', 
                             'url', 'migdar_id', 'item_type', 'first_ccl_query', 'marc_856']

In [4]:
from dataflows import Flow, printer, load, dump_to_path, update_resource
from tabulator import Stream
from openpyxl import load_workbook
import tempfile
import logging
import requests
import os

def process_row(row):
    return {k: str(row[k]) if row.get(k) else '' for k in SEARCH_IMPORT_FIELD_NAMES}


def sheet_iterator(first_row, header_row, stream_iter, filename, sheet_name, stream):
    def inner_sheet_iterator():
        if first_row:
            yield process_row(dict(zip(header_row, first_row)))
        for row in stream_iter:
            yield process_row(dict(zip(header_row, row)))
    for i, row in enumerate(inner_sheet_iterator()):
        if not row['migdar_id']:
            if row['title']:
                print('    {}/{}#{}: missing migdar_id'.format(filename, sheet_name, i))
        else:
            yield row
    stream.close()


def load_from_gdrive_files(rows):
    if rows.res.name == 'search_import_index':
        for row_index, row in enumerate(rows, start=1):
#             if row_index !=5:
#                 continue
            file_url = f"https://migdar-internal-search.odata.org.il/__data/search_import/{row['name']}"
            print(file_url)
            with tempfile.NamedTemporaryFile('w+b', suffix='.xlsx') as temp_file:
                with migdar_session.get(file_url, stream=True) as response:
                    for chunk in response.iter_content():
                        temp_file.write(chunk)
                temp_file.flush()
                wb = load_workbook(temp_file.name)
                for sheet_number, sheet_name in enumerate(wb.sheetnames, start=1):
                    if 'deleted' in sheet_name.strip().lower():
                        continue
                    stream = Stream(temp_file.name, sheet=sheet_name)
                    stream.open()
                    print('#{}.{}/{}: loading sheet'.format(row_index, row['name'], sheet_name))
                    stream_iter = stream.iter()
                    try:
                        first_row = next(stream_iter)
                    except StopIteration:
                        first_row = None
                    if first_row:
                        if 'migdar_id' not in first_row and sheet_number > 1:
                            header_row = first_sheet_header_row
                        else:
                            header_row = first_row
                            first_row = None
                            if sheet_number == 1:
                                first_sheet_header_row = header_row
                        # print(header_row)
                        # for k in header_row:
                        #     if k and k not in [f['name'] for f in schema['fields']]:
                                # print('found field: {}'.format(k))
                        #         field_type = 'string'
                        #         schema['fields'].append({'name': k, 'type': field_type})
                        yield from sheet_iterator(first_row, header_row, stream_iter, row['name'], sheet_name, stream)
    else:
        yield from rows
                            

Flow(
    load('https://migdar-internal-search.odata.org.il/__data/search_import/index.csv', 
         encoding='utf-8', http_session=migdar_session),
    update_resource('index', name='search_import_index', path='search_import_index.csv'),
    load_from_gdrive_files,
    update_resource('search_import_index', name='search_import', path='search_import.csv',
                schema={'fields': [{'name': n,  'type': 'string'} for n in SEARCH_IMPORT_FIELD_NAMES]}),
    printer(num_rows=1, tablefmt='html', fields=['migdar_id', 'pubyear', 'title']),
    dump_to_path('data/search_import_from_gdrive')
).process()[1]

https://migdar-internal-search.odata.org.il/__data/search_import/2018-10-15- ORLY- Gender in conflict + mainstreaming+ based analysis +Income distribution.xlsx
#1.2018-10-15- ORLY- Gender in conflict + mainstreaming+ based analysis +Income distribution.xlsx/מקוטלגים: loading sheet
https://migdar-internal-search.odata.org.il/__data/search_import/2018-10-16- ORLY - Feminism and Science_HEB.xlsx
#2.2018-10-16- ORLY - Feminism and Science_HEB.xlsx/מקוטלגים: loading sheet
https://migdar-internal-search.odata.org.il/__data/search_import/2018-08-30- ORLY - Sex discriminationHEB.xlsx
#3.2018-08-30- ORLY - Sex discriminationHEB.xlsx/מקוטלגים: loading sheet
https://migdar-internal-search.odata.org.il/__data/search_import/2018-08-29- Talia - Employee rights_heb.xlsx
#4.2018-08-29- Talia - Employee rights_heb.xlsx/Sheet1: loading sheet
https://migdar-internal-search.odata.org.il/__data/search_import/2018-08-10- Havatzelet- Feminsm  HEB.xlsx
#5.2018-08-10- Havatzelet- Feminsm  HEB.xlsx/מקוטלגים: lo

#,title (string),pubyear (string),migdar_id (string)
1,"""<<ה>>חצר הנשית"" : תיעוד והמשגה של פרקטיקה ביקורתית בעבודה סוציאלית עם נערות ונשים צעירות ביפו /","תשס""ט 2009.",38652
2,הבדלים בין גברים לנשים בפתרון קונפליקט ובדיאלוג חיובי בתקופת המעבר להורות : הקשר עם בריאות נפשית ושביעות רצון מהזוגיות /,2008,38440
...,,,
2052,חצויה /,"2015 תשע""ה.",2016


{'count_of_rows': 2052,
 'bytes': 2603687,
 'hash': '323f4e7f4d752d8f1c668fac966cf706',
 'dataset_name': None}

## Join with the source data

The GDrive data contains the user-entered data, this joins it with the source data and full marc data

In [5]:
from dataflows import Flow, load, join, printer, set_type

Flow(
    load('data/search_import_from_gdrive/datapackage.json', resources=['search_import']),
    load('data/search_results/unique_records.csv', resources=['unique_records']),
    set_type('migdar_id', type='string', resources=['unique_records', 'search_import']),
    join(source_name='search_import', source_key=['migdar_id'],
         target_name='unique_records', target_key=['migdar_id'],
         fields={f'gd_{field}': {'name': field} for field in SEARCH_IMPORT_FIELD_NAMES},
         full=False),
    printer(tablefmt='html', num_rows=1, fields=['title', 'migdar_id', 'gd_Life Domains']),
    dump_to_path('data/unique_records_full')
).process()[1]

#,title (string),migdar_id (string),gd_Life Domains (string)
1,הפסקות הריון על פי החוק : 2003-1990 /,178,Health
2,הפסקות הריון על פי החוק : 2004-1990 /,186,Health
...,,,
389,The enigma woman the death sentence of Nellie May Madison /,2993,Gender Violence


{'count_of_rows': 389,
 'bytes': 2754961,
 'hash': '1ea8cfc5ffc6e950e3691e827c4f63e2',
 'dataset_name': None}

## Join with the search app data

**there isn't any data yet from the search app (Oct 23, 2018)**

In [6]:
from tabulator import Stream
from dataflows import Flow, printer, checkpoint, load, dump_to_path, update_resource, set_type, join

!{'rm -rf .checkpoints/gdrive_search_app_source_data'}

def load_search_app_data():
    with Stream('https://migdar-internal-search.odata.org.il/__data/search_app/index.csv', 
                http_session=migdar_session) as index_stream:
        for i, row in enumerate(index_stream.iter()):
#             if i > 5:
#                 continue
            search_id = row[4]
            print(f"#{i}. {search_id} ({row[0]}/{row[1]})")
            url = f'https://migdar-internal-search.odata.org.il/__data/search_app/{search_id}/records.csv'
            with Stream(url, headers=1, http_session=migdar_session) as data_stream:
                for rownum, row in enumerate(data_stream.iter(keyed=True)):
                    row['migdar_id'] = f'{search_id}-{rownum}'
                    yield row

Flow(
    load('data/search_import_from_gdrive/datapackage.json', resources=['search_import']),
    load_search_app_data(),
    update_resource('res_2', name='search_app_records', path='search_app_records.csv'),
    checkpoint('gdrive_search_app_source_data'),
    join(source_name='search_import', source_key=['migdar_id'],
         target_name='search_app_records', target_key=['migdar_id'],
         fields={f'gd_{field}': {'name': field} for field in SEARCH_IMPORT_FIELD_NAMES},
         full=False),
    printer(tablefmt='html', num_rows=1, fields=['migdar_id']),
    dump_to_path('data/app_records_full')
).process()[1]

saving checkpoint to: .checkpoints/gdrive_search_app_source_data
#0. 0fb41d0f8f2d4fdbb444b5605ace15f2 (asdfisjdofijsadfasdf/heb)
#1. becab2849d1a403fba5e3a73d6dfa77e (asdfisjdofijsadfasdf/heb)
#2. 83596544daf448caafdead9409f953a8 (discrimination against black women/heb)


#3. d05c183d5ba548dea3e649fdd738ca54 (discrimination/heb)
#4. 42ce23d9dc924e1ab2ce856990399180 (safasdfasdfsadf/heb)
#5. 56cf98417e0e4884ba7d44c4882f687a (adoption black women/heb)
#6. b07130541dd74e1ba33ad1130279721b (adoption women/heb)
#7. 75b8f2db4f9a45b78fc522dbabcbf80f (abortion/arb)
#8. 5a63c01f0717490abf593130675cba6d (discrimination against women/heb)
#9. da40ead706194a9baf20a9e77c1f5549 ("נשים"/heb)
#10. 0791e557229146448975e81206350af9 ("אפליה נגד נשים"/heb)
#11. 72b0fca7a3ce47cc99b46046054f1891 (discrimination/heb)
#12. 2f0b05980d164dce96daf49459e2ee0e ("אפליה נגד נשים פלשתינאיות"/heb)
#13. c254f0d6249c48478db58d31869bb892 ("מחשבים"/heb)
#14. cec11054239e4822933db80b86d80312 ("מחשבים"/heb)
#15. d5225b0fbf0c483492864d0763085a89 ("הדרה של נשים"/heb)
#16. eb78226b2a5946fea12fd723facebdb5 ("הדרה של נשים"/heb)
#17. 14ab8e3e925347d8acf003579fb685aa ("בדואיות"/arb)
#18. 58591008756d40769448495d607d81bc ("בדואיות"/arb)
#19. c626a9bc80314103b05eda44f3f039bc ("אפליה נשים"/heb)
#20. d

checkpoint saved: gdrive_search_app_source_data


{'count_of_rows': 0,
 'bytes': 4996,
 'hash': '215e660d2db2c2e316aecdbeb771453d',
 'dataset_name': None}